In [ ]:
import os, time, random, logging , datetime , cv2 , csv , subprocess
import numpy as np

import tensorflow as tf
print("tf",tf.version.VERSION)
from tensorflow import keras

from utils import globo ,  xdv , tfh5

In [ ]:
''' GPU CONFIGURATION '''

tfh5.set_tf_loglevel(logging.ERROR)
tfh5.tf.debugging.set_log_device_placement(False) #Enabling device placement logging causes any Tensor allocations or operations to be printed.
tfh5.set_memory_growth()
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"

In [ ]:
''' TRAIN & VALDT '''
#train_fn, train_labels, train_tot_frames, valdt_fn, valdt_labels , valdt_tot_frames = xdv.train_valdt_files(tframes=True)
train_fn, train_labels, valdt_fn, valdt_labels = xdv.train_valdt_files()

update_index_train = range(0, len(train_fn))
update_index_valdt = range(0, len(valdt_fn))

In [ ]:
''' CONFIGS '''

train_config = {
    "target_size":[120,160],
    "frame_max_A":8000,
    "target_fps":12,
    
    "ativa" : 'leakyrelu',
    "optima" : 'sgd',
    "batch_type" : 0,   # =0 all batch have frame_max or video length // =1 last batch has frame_max frames // =2 last batch has no repetead frames
    "frame_max" : '4000',
    "ckpt_start" : f"{0:0>8}",  #used in train_model: if 00000000 start from scratch, else start from ckpt with config stated
    "epochs" : 30
}


In [ ]:
class VideoDataGenerator(keras.utils.Sequence):
    def __init__(self, video_paths, labels, batch_size, n_frames, target_fps, shuffle=True):
        self.video_paths = video_paths
        self.labels = labels
        self.batch_size = batch_size
        self.n_frames = n_frames
        self.target_fps = target_fps
        self.shuffle = shuffle
        self.on_epoch_end()

        self.frame_step = 2
    
    def skip_ms(self,cap):
        start_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        
        while True:
            success = cap.grab()

            curr_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)

            if not success or curr_frame - start_frame >= self.frame_step:
                break
        
        if not success:
            return success, None, start_frame + self.frame_step

        success, image = cap.retrieve()
        
        return success, image, curr_frame  
    
    
    def __len__(self):
        return int(np.floor(len(self.video_paths) / self.batch_size))

    def __getitem__(self, index):
        start_index = index * self.batch_size
        end_index = (index + 1) * self.batch_size
        batch_video_paths = self.video_paths[start_index:end_index]
        batch_labels = self.labels[start_index:end_index]

        X, y = self.__data_generation(batch_video_paths, batch_labels)

        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            temp = list(zip(self.video_paths, self.labels))
            np.random.shuffle(temp)
            self.video_paths, self.labels = zip(*temp)

    def __data_generation(self, batch_video_paths, batch_labels):
        X = []
        y = []

        for video_path, label in zip(batch_video_paths, batch_labels):
            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            original_fps = int(cap.get(cv2.CAP_PROP_FPS))
            frame_step = int(original_fps / self.target_fps)

            if label == 0 and total_frames > self.n_frames :
                start_index = random.randint(0, total_frames - self.n_frames)
                end_index = start_index + self.n_frames
                cap.set(cv2.CAP_PROP_POS_FRAMES, start_index)
            else: 
                start_index = 0
                end_index = total_frames

            frames = []
            for i in range(start_index, end_index, frame_step):
                if len(frames) == self.n_frames:
                    break
                cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                ret, frame = cap.read()
                if ret:
                    frame = cv2.resize(frame, (160, 120))
                    frames.append(frame)

            cap.release()
            X.append(frames)
            y.append(label)

        X = np.array(X, dtype=np.float32) / 255.0
        y = np.array(y, dtype=np.float32)

        return X, y

In [ ]:
batch_size = 2
n_frames = 4000  # Number of frames to extract from each video
target_fps = 12  # Target FPS for extracting frames

model = tfh5.form_model(train_config)

data_gen = VideoDataGenerator(train_fn[:4], train_labels[:4], batch_size, n_frames, target_fps)
model.fit(data_gen, epochs=1)

In [ ]:
class FrameGenerator(keras.utils.Sequence):
    def __init__(self, vpath_list, label_list, batch_size, frame_max_A , fps, in_height, in_width):
        
        self.vpath_list = vpath_list
        self.label_list = label_list
        
        
        self.batch_size = batch_size
        self.frame_max_A = frame_max_A
        self.fps = fps
        self.in_height = in_height
        self.in_width = in_width
        
        self.len_vpath_list = len(self.vpath_list)
        self.indices = np.arange(self.len_vpath_list)

        self.frame_step = 2 
    
    
    def skip_ms(self,cap):
        start_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        
        while True:
            success = cap.grab()

            curr_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)

            if not success or curr_frame - start_frame >= self.frame_step:
                break
        
        if not success:
            return success, None, start_frame + self.frame_step

        success, image = cap.retrieve()
        
        return success, image, curr_frame        
        
        
    def __len__(self):
        print("__len__",int(np.ceil(self.len_vpath_list / float(self.batch_size))))
        return int(np.ceil(self.len_vpath_list / float(self.batch_size)))

           
    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size : (idx+1) * self.batch_size]
        
        batch_frames = np.zeros((len(batch_indices),), dtype=np.object)
        #batch_frames = np.zeros((len(batch_indices), self.frame_max_A, self.in_height, self.in_width, 3), dtype=np.float32)
        
        batch_labels = np.zeros((len(batch_indices),), dtype=np.int32)
        
        for i, index in enumerate(batch_indices):
            vpath = self.vpath_list[index]
            #tframes = self.tframes_list[i]
            label = self.label_list[index]
        
        
            video = cv2.VideoCapture(vpath)
            tframes = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
            print(vpath , label , tframes)
            
            if label == 0 and tframes > self.frame_max_A :
                start_index = random.randint(0, tframes - self.frame_max_A)
                end_index = start_index + self.frame_max_A
                video.set(cv2.CAP_PROP_POS_FRAMES, start_index)
            else: 
                start_index = 0
                end_index = tframes
            
            print("sstart_index,end_index",start_index , end_index)
            
            frames = np.zeros((end_index - start_index, self.in_height, self.in_width, 3), dtype=np.float32)
            curr_frame = 0
            success, frame = video.read()
            
            for j in range(end_index - start_index):
                
                if not success or curr_frame > end_index: break
                
                frame = cv2.resize(frame, (self.in_width, self.in_height))
                frame = np.array(frame)/255.0
                frames[j] = frame
                
                #batch_frames[idx][j] = frame
                
                
                #cv2.imshow('ff', frame)
                #key = cv2.waitKey(int(1000/12))
                #if key == ord('q'): break  # quit
                #if key == ord(' '):  # pause
                #    while True:
                #        key = cv2.waitKey(1)
                #        if key == ord(' '):break
                
                
                success, frame, curr_frame = self.skip_ms(video)
            
            batch_frames[i] = frames
            batch_labels[i] = label

            #batch_labels[idx] = label

        print(np.shape(batch_frames),np.shape(batch_labels))
        return batch_frames, batch_labels  

In [ ]:
train_generator = FrameGenerator(train_fn[:4], train_labels[:4], 2, 8000, 12, 120, 160)

#bf , bl = train_generator.getitem(0)
model = tfh5.form_model(train_config)

model.fit(train_generator, epochs=1 , use_multiprocessing = True , workers = 8)

NO CLASS

In [ ]:
def video_generator(video_paths, labels, fps, window_size, in_height, in_width):
    
    
    def skip_ms(cap,frame_step):
        start_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        
        while True:
            success = cap.grab()

            curr_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)

            if not success or curr_frame - start_frame >= frame_step:
                break
        
        if not success:
            return success, None, start_frame + frame_step

        success, image = cap.retrieve()
        
        return success, image, curr_frame    
    
    
    #while True:
    # Shuffle the video paths and labels
    zipped = list(zip(video_paths, labels))
    random.shuffle(zipped)
    video_paths, labels = zip(*zipped)
    
    # Loop over the video paths and labels
    #for i in range(len(video_paths)):
    video_path = video_paths[0]
    label = labels[0]
    
    video = cv2.VideoCapture(video_path)
    tframes = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(video_path , label , tframes)
    
    if label == 0 and tframes > 4000 :
        start_index = random.randint(0, tframes - 4000)
        end_index = start_index + 4000
        video.set(cv2.CAP_PROP_POS_FRAMES, start_index)
    else : 
        start_index = 0
        end_index = tframes
    
    print(start_index , end_index)
    
    frames = []
    curr_frame = 0
    
    success, frame = video.read()
    while 1:
        
        if not success or curr_frame > end_index: break
        
        frame = cv2.resize(frame, (in_width, in_height))
        frame = np.array(frame)/255.0
        frames.append(frame)
        
        cv2.imshow('ff', frame)
        key = cv2.waitKey(int(1000/12))
        if key == ord('q'): break  # quit
        if key == ord(' '):  # pause
            while True:
                key = cv2.waitKey(1)
                if key == ord(' '):break
        
         
        success, frame, curr_frame = skip_ms(video,2)
        
    #batch_frames.append(frames)
    #batch_labels.append(label)
    #
    #batch_frames = np.array(batch_frames) / 255.0
    #batch_labels = np.array(batch_labels)
    
    X = np.array(frames)
    
        
    return np.expand_dims(X,0) , np.array([label])
        

In [ ]:
x , y = video_generator(train_fn,train_labels,12,8000,120,160)

print( np.shape(x) , np.shape(y))

ORIGINAL

In [ ]:
""" INPUT DATA"""

in_height = 120; in_width = 160

def input_train_video_data(file_name):
    print("\n\ninput_train_video_data\n")
    
    #file_name = 'C:\\Bosch\\Anomaly\\training\\videos\\13_007.avi'
    
    video = cv2.VideoCapture(file_name)
    total_frame = video.get(cv2.CAP_PROP_FRAME_COUNT)
    
    divid_no = 1
    
    frame_max = train_config["frame_max"]
    
    # define the nmbers of batchs to divid atual video (divid_no)
    if total_frame > int(frame_max):
        total_frame_int = int(total_frame)
        if total_frame_int % int(frame_max) == 0:
            divid_no = int(total_frame / int(frame_max))
        else:
            divid_no = int(total_frame / int(frame_max)) + 1
        
    batch_no = 0
    batch_frames = []
    batch_frames_flip = []
    counter = 0
    
    # gets random batch w\ frame max lenght 
    if 'Normal' in file_name:
        print("\n\nNORMAL\n\n")
        if divid_no != 1:
            slice_no = int(random.random()*divid_no)
            passby = 0
            if slice_no != divid_no - 1:
                while video.isOpened and passby < int(frame_max) * slice_no:
                    passby += 1
                    success, image = video.read()
                    if success == False:
                        break
            else:
                while video.isOpened and passby < total_frame - int(frame_max):
                    passby += 1
                    success, image = video.read()
                    if success == False:
                        break

    while video.isOpened:               
        success, image = video.read()
        if success == False:
            break

        image = cv2.resize(image, (in_width, in_height))
        image_flip = cv2.flip(image, 1)
        
        image_array = np.array(image)/255.0
        image_array_flip = np.array(image_flip)/255.0
        
        batch_frames.append(image_array)
        batch_frames_flip.append(image_array_flip)
        
        counter += 1
        if counter > int(frame_max):
            break
            
    video.release()
    batch_frames = np.array(batch_frames)
    #print(batch_frames.shape)
        
    return np.expand_dims(batch_frames,0), np.expand_dims(batch_frames_flip, 0), total_frame



def generate_input(data,update_index,validation):
    
    data_var_name = [k for k, v in globals().items() if v is data][0]
    print("\n\nGENERATE_INPUT FOR",data_var_name,\
        '\n\tupdate_index len = ',len(update_index),\
        '\n\tdata len = ',len(data))
    
    loop_no = 0
    while 1:
        index = update_index[loop_no]
        loop_no += 1
        print("\n",data_var_name," index",index," loop_no",loop_no)
        if loop_no == len(data):loop_no= 0
        
        
        batch_frames, batch_frames_flip, total_frames = input_train_video_data(data[index])
        print("\n\t",data_var_name,"data[",index,"]=",data[index],"\n\ttotal_frames=",total_frames,"\n\tbatch_frames.shape=",batch_frames.shape,"\n")
        #if batch_frames.ndim != 5:
        #   break

        
        if not validation:
            #batch_frames
            if 'label_A' in data[index]: yield batch_frames, np.array([0])   #normal
            else: yield batch_frames, np.array([1])   #abnormal
            
            #batch_frames_flip
            if 'label_A' in data[index]: yield batch_frames_flip, np.array([0])  #normal
            else: yield batch_frames_flip, np.array([1])  #abnormal
        else:
            #batch_frames
            if 'label_A' in data[index]: yield batch_frames, np.array([0])   #normal
            else: yield batch_frames, np.array([1])   #abnormal
                
    print("\nloop_no=",loop_no)